In [1]:
# Installation of any non native supported libraries

# !pip install beautifulsoup4
# !pip install wget

In [1]:
# Import necessary libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
import wget
import os
import re 
import openpyxl
import pandas as pd
from datetime import datetime
import calendar

In [177]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

## Download Data

In [178]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

In [179]:
Downloaded_data = []

# Current Date
current_date = datetime.today()
current_year, current_month = int(current_date.strftime('%Y')), int(current_date.strftime('%m'))
current_month = current_month - 2 # Account for delay uploading data by data.gov.uk
start_year = current_year


while current_year >= 2018:
    # Webpage
    URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, "html.parser")
    file_name = '{}/UK Data/{} {} UK Monthly Statistics of Building Materials and Components.xlsx'.format(download_destination, current_year,calendar.month_name[current_month])
    print('{} {}'.format(current_year, calendar.month_name[current_month]))
    
    for a in soup.find_all("a", href=lambda href: href.lower() and str(current_year).lower() in href and str(calendar.month_name[current_month]).lower() in href.lower()):
        l = a['href']
        
    if not l.endswith('.xlsx'):
        r = requests.get(l) 
        soup = BeautifulSoup(r.content, "html.parser")
        for a in soup.find_all("a", href=lambda href: href.lower() and str(current_year).lower() in href and str(calendar.month_name[current_month]).lower() in href.lower() and '.xlsm' in href.lower(), limit=1):
            l = a['href']
            
    # Download link
    print(l)
    r = requests.get(l) 
    with open(file_name, 'wb') as f:
        f.write(r.content)     
    Downloaded_data.append(file_name)
    
    current_year = current_year - 1
    
print('')
print('100% |■■■■■■■■■■■■■■■■■■■■■■■■■■■|')
print('Total files downloaded: ' + str(len(Downloaded_data)))

2023 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx
2022 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1065392/22-cs4_-_Construction_Building_Materials_-_Tables_March_2022.xlsx
2021 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/976157/21-cs4_-_Construction_Building_Materials_-_Tables_March_2021.xlsm
2020 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/876765/20-cs4_-_Construction_Building_Materials_-_Tables_March_2020.xlsm
2019 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/791320/19-cs4_-_Construction_Building_Materials_-_Excel_Tables_March_2019.xlsm
2018 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachm

In [241]:
# Creating Empty DataFrame for storing compiled data
UK_Data_Masterfile = pd.DataFrame()
UK_Data_Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

In [242]:
current_year = 2023
for wb in Downloaded_data[:3]:
    print('{} {}'.format(current_year, calendar.month_name[current_month]))
    print(wb)
    if current_year >= 2020:
        # Define variable to load the workbook
        wb = openpyxl.load_workbook(wb)
        sheets = wb.sheetnames
        ws = wb[sheets[3]]
        df = pd.DataFrame(ws.values)

        # Delete filler rows
        if current_year < 2020:        
            rows_to_delete = [0,1,2,3,4,5,7,8,9,10,12,13,19,20,24,25,32,33,41,42,47,48,58]
        else:
            rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
        df = df.drop(index=rows_to_delete)
        df.reset_index(drop=True, inplace=True) # Reset the index 
        df = df.drop(df.tail(15).index)

        # Set Column Headers
        df = df.T.set_index(0).T # Assign top row as column headers
        df.columns.values[0] = 'Materials' # Label index column
        df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

        # Change Header Format to YYYY-Month
        curr_month = current_month
        curr_year = current_year
        col_list = [col for col in df.columns[::-1]]
        new_col = []
        for c in col_list[:-1]:
            curr_month = curr_month - 1
            if curr_month == 0:
                curr_year = curr_year - 1
                curr_month = 12  
            c = str(curr_year) + ' ' + calendar.month_name[curr_month]
            new_col.append(c)    
        new_col.append('Materials') 
        new_col.reverse()
        df.columns = new_col
        df1 = df
        
        if current_year != 2023:
            # Update Masterfile using concat to append new data
            UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
            dfx = pd.merge(UK_Data_Masterfile, df1, left_index=True, right_index=True, how='inner', suffixes=('', '_y'))
            dfx = dfx.drop(dfx.filter(regex='_y$').columns, axis=1)
            
            index = dfx.columns[0]
            new_data = dfx.columns[-12::]
            old_data = dfx.columns[1:-12]
            reordered = [index , *new_data , *old_data]
            new_df = dfx[reordered]
            new_df.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
            print('2')
        else:
            # Update Masterfile using concat to append new data
            UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
            new_df = pd.merge(df1, UK_Data_Masterfile, left_index=True, right_index=True, how='outer')
            new_df.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
            print("1")

    else:
        print('')
        
    current_year = current_year - 1

2023 March
/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/UK Data/2023 March UK Monthly Statistics of Building Materials and Components.xlsx
1
2022 March
/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/UK Data/2022 March UK Monthly Statistics of Building Materials and Components.xlsx
2
2021 March
/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/UK Data/2021 March UK Monthly Statistics of Building Materials and Components.xlsx
2
2020 March
/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/UK Data/2020 March UK Monthly Statistics of Building Materials and Components.xlsx
2
2019 March
/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/UK Data/2019 March UK Monthly Statistics of Building Materials and Components.xlsx

2018 March
/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_A

In [234]:
# Define variable to load the workbook
wb = openpyxl.load_workbook(Downloaded_data[3])
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,PRICES,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,PRICES
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Table 2: Price Indices of Construction Materials,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,United Kingdom,None,None,None,None,None,None,2010=100,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2010=100
5,None,2019,,,,,,,None,None,,None,,None,,None,,None,2020,None,,None,None
6,None,Feb,Mar,Apr,May,Jun,Jul,Aug,None,None,Sep,None,Oct,None,Nov,None,Dec,None,Jan,None,Feb,None,None
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,AGGREGATES,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,p,None,p,None,AGGREGATES
9,Crushed rock - including levy,106.2,106.1,106.1,106.4,107.7,108.2,108.5,None,None,107.2,None,106.8,None,106.8,None,106.8,None,106.8,None,106.8,None,Crushed rock - including levy


In [235]:
# Delete filler rows
rows_to_delete = [0,1,2,3,4,5,7,8,9,10,11,12,13,14,19,20,24,25,32,33,41,42,47,48,58]
df = df.drop(index=rows_to_delete)

In [236]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,None,Feb,Mar,Apr,May,Jun,Jul,Aug,None,None,Sep,None,Oct,None,Nov,None,Dec,None,Jan,None,Feb,None,None
1,Sand & gravel - including levy,121,121,121,121,121,121,121,None,None,121,None,121,None,121,None,121,None,121,None,121,None,Sand & gravel - including levy
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,CEMENT AND CONCRETE,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CEMENT AND CONCRETE
4,Cement,124.5,124.5,124.7,124.9,124.8,125.1,124.8,None,None,124.7,None,124.5,None,124.3,None,124.2,None,127,None,127.4,None,Cement
5,Ready-mixed concrete **,121.8,121.6,121.7,122.5,122.3,122.6,122.3,None,None,122.1,None,121.7,r,121.9,None,121.7,None,121.8,None,122,None,Ready-mixed concrete **
6,Pre-cast concrete products,128.6,128.8,129.8,129.7,129.5,130,129.5,None,None,129.6,None,129.6,None,129.5,None,129.2,None,129.9,None,130.6,None,Pre-cast concrete products
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,CLAY PRODUCTS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CLAY PRODUCTS
9,All Bricks,c,c,c,c,c,c,c,None,None,c,None,c,None,c,None,c,None,c,None,c,None,All Bricks


In [256]:
current_year = 2023
# Define variable to load the workbook
wb = openpyxl.load_workbook(Downloaded_data[0])
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)

# Delete filler rows
rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
df = df.drop(index=rows_to_delete)
df.reset_index(drop=True, inplace=True) # Reset the index 
df = df.drop(df.tail(15).index)

# Set Column Headers
df = df.T.set_index(0).T # Assign top row as column headers
df.columns.values[0] = 'Materials' # Label index column
df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

# Change Header Format to YYYY-Month
curr_month = current_month
curr_year = current_year
col_list = [col for col in df.columns[::-1]]
new_col = []
for c in col_list[:-1]:
    curr_month = curr_month - 1
    if curr_month == 0:
        curr_year = curr_year - 1
        curr_month = 12  
    c = str(curr_year) + ' ' + calendar.month_name[curr_month]
    new_col.append(c)    
new_col.append('Materials') 
new_col.reverse()
df.columns = new_col
df1 = df
x = df1.iloc[:, 0]

In [257]:
x

1     Gravel, Sand, Clays & Kaolin  - exc Aggregate ...
2                                                   ...
3     Bituminous Mixtures based on Natural & Artific...
4                                               Cement 
5                               Ready-mixed concrete **
6                            Pre-cast concrete products
7             of which : Blocks, bricks, tiles  & fl...
8                     Concrete reinforcing bars (steel)
9                                           All Bricks 
10                                     Ceramic tiles * 
11                                 Ceramic sanitaryware
12                      Imported sawn or planed wood   
13                                     Imported plywood
14                                        Sawn wood ** 
15                                   Particle Board ** 
16                                 Builders woodwork **
17                        of which : Doors & windows **
18                          Fabricated structura

In [259]:
current_year = 2020
# Define variable to load the workbook
wb = openpyxl.load_workbook(Downloaded_data[3])
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)

# Delete filler rows
rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
df = df.drop(index=rows_to_delete)
df.reset_index(drop=True, inplace=True) # Reset the index 
df = df.drop(df.tail(15).index)

# Set Column Headers
df = df.T.set_index(0).T # Assign top row as column headers
df.columns.values[0] = 'Materials' # Label index column
df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

# Change Header Format to YYYY-Month
curr_month = current_month
curr_year = current_year
col_list = [col for col in df.columns[::-1]]
new_col = []
for c in col_list[:-1]:
    curr_month = curr_month - 1
    if curr_month == 0:
        curr_year = curr_year - 1
        curr_month = 12  
    c = str(curr_year) + ' ' + calendar.month_name[curr_month]
    new_col.append(c)    
new_col.append('Materials') 
new_col.reverse()
df.columns = new_col
df2 = df
df2

,Materials,2019 February,2019 March,2019 April,2019 May,2019 June,2019 July,2019 August,2019 September,2019 October,2019 November,2019 December,2020 January,2020 February
1,Crushed rock - including levy,106.2,106.1,106.1,106.4,107.7,108.2,108.5,107.2,106.8,106.8,106.8,106.8,106.8
2,- exclud...,108.3,108.2,108.2,108.5,110.6,111.3,111.7,109.8,109.1,109.1,109.1,109.1,109.1
3,Sand & gravel - including levy,121,121,121,121,121,121,121,121,121,121,121,121,121
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,CEMENT AND CONCRETE,None,None,None,None,None,None,None,None,None,None,None,None,None
6,Cement,124.5,124.5,124.7,124.9,124.8,125.1,124.8,124.7,124.5,124.3,124.2,127,127.4
7,Ready-mixed concrete **,121.8,121.6,121.7,122.5,122.3,122.6,122.3,122.1,121.7,121.9,121.7,121.8,122
8,Pre-cast concrete products,128.6,128.8,129.8,129.7,129.5,130,129.5,129.6,129.6,129.5,129.2,129.9,130.6
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None
10,CLAY PRODUCTS,None,None,None,None,None,None,None,None,None,None,None,None,None


In [163]:
# Update Masterfile using concat to append new data
UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
dfx = pd.merge(df1, UK_Data_Masterfile, left_index=True, right_index=True, how='outer', suffixes=('', '_y'))
dfx

,Materials,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137
6,Pre-cast concrete products,148,148.5,150.5,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8,178.2
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,140.6,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1,161.9
8,Concrete reinforcing bars (steel),169.7,213.8,240.7,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2,185.8
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c


In [146]:
current_year = 2022
# Define variable to load the workbook
wb = openpyxl.load_workbook(Downloaded_data[1])
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)

# Delete filler rows
rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
df = df.drop(index=rows_to_delete)
df.reset_index(drop=True, inplace=True) # Reset the index 
df = df.drop(df.tail(15).index)

# Set Column Headers
df = df.T.set_index(0).T # Assign top row as column headers
df.columns.values[0] = 'Materials' # Label index column
df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

# Change Header Format to YYYY-Month
curr_month = current_month
curr_year = current_year
col_list = [col for col in df.columns[::-1]]
new_col = []
for c in col_list[:-1]:
    curr_month = curr_month - 1
    if curr_month == 0:
        curr_year = curr_year - 1
        curr_month = 12  
    c = str(curr_year) + ' ' + calendar.month_name[curr_month]
    new_col.append(c)    
new_col.append('Materials') 
new_col.reverse()
df.columns = new_col
df2 = df

In [152]:
dfx = pd.merge(df1, df2, left_index=True, right_index=True, how='inner', suffixes=('', '_y'))
dfx.drop(dfx.filter(regex='_y$').columns, axis=1, inplace=True)
index = dfx.columns[0]
new_data = dfx.columns[-12::]
old_data = dfx.columns[1:-12]
reordered = [index , *new_data , *old_data]
print(reordered)

new_df = dfx[reordered]
new_df

,Materials,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February,2021 February,2021 March,2021 April,2021 May,2021 June,2021 July,2021 August,2021 September,2021 October,2021 November,2021 December,2022 January
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7,116.4,112.1,113.3,113.3,113.7,114.5,115.2,115,115,114.6,113.6,120
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164,138.2,122.8,122.3,122.3,122.8,122.8,122.8,124.5,127,127,127,132.6
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9,108.3,108.2,107.6,108.1,109.2,109.3,109.3,108.8,109.2,115.8,118.4,120.1
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6,108.9,109.6,110.1,111.1,111,110.9,110.9,111.3,113.1,113.9,113.9,115.1
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137,100.4,100.5,102,104.5,104.7,105.3,106.1,108.6,108.6,108.7,108.8,113
6,Pre-cast concrete products,148,148.5,150.5,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8,178.2,117.9,117.7,119,126.1,126.3,128,130.3,130.7,135.1,135.6,136.1,146.3
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,140.6,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1,161.9,116.6,116.3,117.8,117.5,117.5,118,118.3,118.9,123.5,124.1,123.8,135.5
8,Concrete reinforcing bars (steel),169.7,213.8,240.7,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2,185.8,123.8,135.9,140.5,147.4,157,166.1,169.4,167.6,167.2,167.3,168,168.6
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c


In [132]:
index

'Materials'

In [226]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
dfx

,Materials,2021 February,2021 March,2021 April,2021 May,2021 June,2021 July,2021 August,2021 September,2021 October,2021 November,2021 December,2022 January,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February,2020 February,2020 March,2020 April,2020 May,2020 June,2020 July,2020 August,2020 September,2020 October,2020 November,2020 December,2021 January
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",116.4,112.1,113.3,113.3,113.7,114.5,115.2,115,115,114.6,113.6,120,123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7,c,c,c,c,c,c,110.6,110.7,110.9,116.8,116.3,115.9
2,...,138.2,122.8,122.3,122.3,122.8,122.8,122.8,124.5,127,127,127,132.6,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164,114.8,114.8,114.8,114.8,117.3,117.5,117.5,117.5,119.1,134.5,134.5,137.1
3,Bituminous Mixtures based on Natural & Artific...,108.3,108.2,107.6,108.1,109.2,109.3,109.3,108.8,109.2,115.8,118.4,120.1,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9,108.7,107.6,107.6,107.6,107.1,107.1,107.1,107.1,106.1,107.2,107.1,105.8
4,Cement,108.9,109.6,110.1,111.1,111,110.9,110.9,111.3,113.1,113.9,113.9,115.1,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6,107.2,107.8,107.8,107.8,107.8,107,107.4,107.9,106.9,107.1,106.8,108.1
5,Ready-mixed concrete **,100.4,100.5,102,104.5,104.7,105.3,106.1,108.6,108.6,108.7,108.8,113,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137,103.7,101.7,101.6,101.7,99.7,99,99.6,101,101.2,101.3,100.7,101.4
6,Pre-cast concrete products,117.9,117.7,119,126.1,126.3,128,130.3,130.7,135.1,135.6,136.1,146.3,148,148.5,150.5,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8,178.2,115.5,115.5,117.5,117.7,117.3,117.2,117.4,117.3,117.3,117.3,117,117.6
7,"of which : Blocks, bricks, tiles & fl...",116.6,116.3,117.8,117.5,117.5,118,118.3,118.9,123.5,124.1,123.8,135.5,138.2,138.5,140.6,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1,161.9,115.5,115.6,116.6,117,116.4,116.1,115.7,115.5,115.6,115.3,115.3,116.3
8,Concrete reinforcing bars (steel),123.8,135.9,140.5,147.4,157,166.1,169.4,167.6,167.2,167.3,168,168.6,169.7,213.8,240.7,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2,185.8,102.4,105.5,104,103.6,101.8,101.5,102.4,103.5,104.5,106.8,110.1,118.7
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,c,95.8,c


In [6]:
# Webpage
URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
r = requests.get(URL) 
soup = BeautifulSoup(r.content, "html.parser")
list_of_urls_found = []

# Creating Empty DataFrame for storing compiled data
UK_Data_Masterfile = pd.DataFrame()
UK_Data_Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

# Current Date
current_date = datetime.today()
current_year, current_month = int(current_date.strftime('%Y')), int(current_date.strftime('%m'))
current_month = current_month - 2 # Account for delay uploading data by data.gov.uk
start_year = current_year


while current_year >= 2015:
    print('{} {}'.format(current_year, calendar.month_name[current_month]))
    
    file_name = '{}/UK Data/{} {} UK Monthly Statistics of Building Materials and Components.xlsx'.format(download_destination, current_year,calendar.month_name[current_month])
    list_of_urls_found = []
    for a in soup.select(','.join('a:contains("{}")'.format(i) for i in calendar.month_name[current_month])):
            list_of_urls_found.append(a['href'])
    for l in list_of_urls_found:
        if re.findall('{}.*{}'.format(calendar.month_name[current_month], current_year).lower(), l.lower()):
            download_link = l
            print(download_link)
            
    if download_link.endswith('.xlsx'):
        # Download link
        r = requests.get(download_link) 
        with open(file_name, 'wb') as f:
            f.write(r.content)
    else:
        if 'https://www.gov.uk/' not in download_link:
            download_link = 'https://www.gov.uk/' + download_link
        # Some links need further digging to extract
        r = requests.get(download_link) 
        soup = BeautifulSoup(r.content, "html.parser")
        file_name = '{}/UK Data/{} {} UK Monthly Statistics of Building Materials and Components.xlsx'.format(download_destination, current_year,calendar.month_name[current_month])
        list_of_urls_found = []
        for a in soup.select(','.join('a:contains("{}")'.format(i) for i in calendar.month_name[current_month])):
            list_of_urls_found.append(a['href'])
        for l in list_of_urls_found:
            if l.endswith('.xlsm'):
                if re.findall('{}.*{}'.format(calendar.month_name[current_month], current_year).lower(), l.lower()):
                    download_link = l
            r = requests.get(download_link) 
            with open(file_name, 'wb') as f:
                f.write(r.content)
    
    # Define variable to load the workbook
    wb = openpyxl.load_workbook(file_name)
    sheets = wb.sheetnames
    ws = wb[sheets[3]]
    df = pd.DataFrame(ws.values)
    
    # Delete filler rows
    rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
    df = df.drop(index=rows_to_delete)
    df.reset_index(drop=True, inplace=True) # Reset the index 
    df = df.drop(df.tail(15).index)
    
    # Set Column Headers
    df = df.T.set_index(0).T # Assign top row as column headers
    df.columns.values[0] = 'Materials' # Label index column
    df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

    # Change Header Format to YYYY-Month
    curr_month = current_month
    curr_year = current_year
    col_list = [col for col in df.columns[::-1]]
    new_col = []
    for c in col_list[:-1]:
        curr_month = curr_month - 1
        if curr_month == 0:
            curr_year = curr_year - 1
            curr_month = 12  
        c = str(curr_year) + ' ' + calendar.month_name[curr_month]
        new_col.append(c)    
    new_col.append('Materials') 
    new_col.reverse()
    df.columns = new_col
    
    
    
    # Append new data to existing database
    if current_year == start_year:
        # Update Masterfile using concat to append new data
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
        Masterfile = pd.concat([UK_Data_Masterfile, df.iloc[:, :-1]])
        Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
    else:
        # Update Masterfile using concat to append new data
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)        
        orginal_labels = UK_Data_Masterfile.iloc[: , :1]
        temp_df = UK_Data_Masterfile.iloc[: , 1:]
        new_df = pd.concat([df.iloc[:, :-1], temp_df], axis=1, join='outer')
        new_df.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
    
    current_year = current_year - 1

2023 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx
2022 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1065392/22-cs4_-_Construction_Building_Materials_-_Tables_March_2022.xlsx
2021 March
https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2021


KeyboardInterrupt: 

In [234]:
list_of_urls_found

['https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx',
 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1065392/22-cs4_-_Construction_Building_Materials_-_Tables_March_2022.xlsx',
 'https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2021',
 'https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2020',
 'https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2019',
 'https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2018',
 'https://www.gov.uk/government/collections/building-materials-and-components-monthly-statistics-2012',
 'https://www.gov.uk/government/collections/building-materials-and-components-monthly-statistics-2012',
 'https://www.gov.uk/government/collectio

In [ ]:
download_link

In [235]:
for l in list_of_urls_found:
    if re.findall('{}.*{}'.format(calendar.month_name[current_month], current_year).lower(), l.lower()):
        download_link = l
        print(download_link)

https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2020


In [228]:
list_of_urls_found

['#content',
 '/help/cookies',
 '/help/cookies',
 '/help/cookies',
 '/browse',
 '/search',
 '/browse/births-deaths-marriages',
 '/browse/business',
 '/browse/childcare-parenting',
 '/browse/citizenship',
 '/browse/justice',
 '/browse/disabilities',
 '/browse/driving',
 '/browse/education',
 '/browse/environment-countryside',
 '/browse/housing-local-services',
 '/browse/tax',
 '/browse/abroad',
 '/browse/visas-immigration',
 '/browse/working',
 '/government/organisations',
 '/search/guidance-and-regulation',
 '/search/research-and-statistics',
 '/search/policy-papers-and-consultations',
 '/search/transparency-and-freedom-of-information-releases',
 '/cost-of-living',
 '/guidance/getting-the-energy-bills-support-scheme-discount',
 '/find-a-job',
 '/coronavirus',
 '/sign-in-universal-credit',
 '/business-and-industry',
 '/government/organisations/department-for-business-energy-and-industrial-strategy',
 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_

In [194]:
current_month = 3
current_year = 2021
# Webpage
URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
r = requests.get(URL) 
soup = BeautifulSoup(r.content, "html.parser")
list_of_urls_found = []


list_of_urls_found = []
for a in soup.select(','.join('a:contains("{}")'.format(i) for i in calendar.month_name[current_month])):
        list_of_urls_found.append(a['href'])
for l in list_of_urls_found:
    if re.findall('{}.*{}'.format(calendar.month_name[current_month], current_year).lower(), l.lower()):
        download_link = l
        print(download_link)

https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2021


In [187]:
new_df

,Materials,2021 February,2021 March,2021 April,2021 May,2021 June,2021 July,2021 August,2021 September,2021 October,...,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",116.4,112.1,113.3,113.3,113.7,114.5,115.2,115,115,...,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5
2,...,138.2,122.8,122.3,122.3,122.8,122.8,122.8,124.5,127,...,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,108.3,108.2,107.6,108.1,109.2,109.3,109.3,108.8,109.2,...,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1
4,Cement,108.9,109.6,110.1,111.1,111,110.9,110.9,111.3,113.1,...,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3
5,Ready-mixed concrete **,100.4,100.5,102,104.5,104.7,105.3,106.1,108.6,108.6,...,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137
6,Pre-cast concrete products,117.9,117.7,119,126.1,126.3,128,130.3,130.7,135.1,...,150.5,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8
7,"of which : Blocks, bricks, tiles & fl...",116.6,116.3,117.8,117.5,117.5,118,118.3,118.9,123.5,...,140.6,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1
8,Concrete reinforcing bars (steel),123.8,135.9,140.5,147.4,157,166.1,169.4,167.6,167.2,...,240.7,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2
9,All Bricks,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,c,c


In [164]:
UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
temp_df = UK_Data_Masterfile.iloc[: , 1:]
new_df = pd.concat([df, temp_df], axis=1, join='outer')
new_df.head()

,Materials,2023 February,2023 January,2022 December,2022 November,2022 October,2022 September,2022 August,2022 July,2022 June,...,2022 December,2022 November,2022 October,2022 September,2022 August,2022 July,2022 June,2022 May,2022 April,2022 March
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",141.7,142.5,142.7,142.7,136.9,139,137.6,137.6,139.5,...,142.7,142.7,136.9,139,137.6,137.6,139.5,139.3,132.7,123.8
2,...,164,164,164,164,161.5,161.5,161.5,162.2,165.3,...,164,164,161.5,161.5,161.5,162.2,165.3,166.2,137.3,130.4
3,Bituminous Mixtures based on Natural & Artific...,139.9,140.1,139.5,139.5,141.7,142.3,137.7,135.8,128.8,...,139.5,139.5,141.7,142.3,137.7,135.8,128.8,127.9,126.4,122.3
4,Cement,143.6,143.3,138.3,134.4,132.6,132.3,131.1,130.7,127.2,...,138.3,134.4,132.6,132.3,131.1,130.7,127.2,124.3,124.3,121.7
5,Ready-mixed concrete **,137,137,136,134.3,132.1,131.3,129.3,128.8,128.5,...,136,134.3,132.1,131.3,129.3,128.8,128.5,126.3,120.2,119.6


In [148]:
df

,Materials,2022 February,2022 January,2021 December,2021 November,2021 October,2021 September,2021 August,2021 July,2021 June,2021 May,2021 April,2021 March,2021 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",126.4,120,113.6,114.6,115,115,115.2,114.5,113.7,113.3,113.3,112.1,116.4
2,...,132.6,132.6,127,127,127,124.5,122.8,122.8,122.8,122.3,122.3,122.8,138.2
3,Bituminous Mixtures based on Natural & Artific...,122.2,120.1,118.4,115.8,109.2,108.8,109.3,109.3,109.2,108.1,107.6,108.2,108.3
4,Cement,120.1,115.1,113.9,113.9,113.1,111.3,110.9,110.9,111,111.1,110.1,109.6,108.9
5,Ready-mixed concrete **,113.4,113,108.8,108.7,108.6,108.6,106.1,105.3,104.7,104.5,102,100.5,100.4
6,Pre-cast concrete products,148.1,146.3,136.1,135.6,135.1,130.7,130.3,128,126.3,126.1,119,117.7,117.9
7,"of which : Blocks, bricks, tiles & fl...",139.1,135.5,123.8,124.1,123.5,118.9,118.3,118,117.5,117.5,117.8,116.3,116.6
8,Concrete reinforcing bars (steel),177.8,168.6,168,167.3,167.2,167.6,169.4,166.1,157,147.4,140.5,135.9,123.8
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c


In [106]:
df.head()

,Materials,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Jan,Feb
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137


In [42]:
URL = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx'
r = requests.get(URL) 
with open(r'/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/UK Data/FIle_new.xlsx', 'wb') as f:
    f.write(r.content)

In [43]:


# Define variable to load the wookbook
wb = openpyxl.load_workbook("/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/file_name.xlsx")
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)

df.head(50)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,PRICES,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,PRICES
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Table 2: Price Indices of Construction Materials,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,United Kingdom,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2015=100
5,None,2022,,None,,None,,None,,None,...,None,,None,,None,2023,None,,None,None
6,None,Feb,Mar,None,Apr,None,May,None,Jun,None,...,None,Nov,None,Dec,None,Jan,None,Feb,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,AGGREGATES,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,p,None,p,None,AGGREGATES
9,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,None,132.7,None,139.3,None,139.5,None,...,None,142.7,None,142.7,None,142.5,None,141.7,None,"Gravel, Sand, Clays & Kaolin - exc Aggregate ..."


In [44]:
rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]

df = df.drop(index=rows_to_delete)
df.reset_index(drop=True, inplace=True) # Reset the index 

In [45]:
df = df.drop(df.tail(15).index)

In [46]:
df.tail(50)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,None,Feb,Mar,None,Apr,None,May,None,Jun,None,...,None,Nov,None,Dec,None,Jan,None,Feb,None,None
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,None,132.7,None,139.3,None,139.5,None,...,None,142.7,None,142.7,None,142.5,None,141.7,None,"Gravel, Sand, Clays & Kaolin - exc Aggregate ..."
2,...,129.4,130.4,None,137.3,None,166.2,None,165.3,None,...,None,164,r,164,None,164,None,164,None,...
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,None,126.4,None,127.9,None,128.8,None,...,r,139.5,r,139.5,None,140.1,None,139.9,None,Bituminous Mixtures based on Natural & Artific...
4,Cement,119.9,121.7,None,124.3,None,124.3,None,127.2,None,...,None,134.4,None,138.3,None,143.3,None,143.6,None,Cement
5,Ready-mixed concrete **,117.2,119.6,r,120.2,r,126.3,r,128.5,r,...,r,134.3,r,136,None,137,None,137,None,Ready-mixed concrete **
6,Pre-cast concrete products,148,148.5,None,150.5,None,161.6,None,162.2,None,...,r,169.6,r,170.2,None,172.8,None,178.2,None,Pre-cast concrete products
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,None,140.6,None,148.3,None,147.7,None,...,None,150.8,r,151.7,None,156.1,None,161.9,None,"of which : Blocks, bricks, tiles & fl..."
8,Concrete reinforcing bars (steel),169.7,213.8,None,240.7,None,242.2,None,239.8,None,...,None,199.9,None,199.6,p,185.2,p,185.8,p,Concrete reinforcing bars (steel)
9,All Bricks,c,c,None,c,None,c,None,c,None,...,None,c,None,c,None,c,None,c,None,All Bricks


In [47]:
df = df.T.set_index(0).T # Assign top row as column headers

In [48]:
df.columns.values[0] = 'Materials' # Label index column

In [49]:
df = df.drop([col for col in df.columns if not col], axis=1)

In [50]:
df.head()

,Materials,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Jan,Feb
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137


In [51]:
dfx = df

In [52]:
from datetime import datetime
current_date = datetime.today()
current_year = int(current_date.strftime('%Y'))
current_month = int(current_date.strftime('%m'))
print(current_month)

5


In [53]:
import calendar
d = dict(enumerate(calendar.month_name))
print(d)

{0: '', 1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}


In [54]:
i = 3
calendar.month_name[i]

'March'

In [55]:
# format dates correctly

col_list = [col for col in dfx.columns[::-1]]
new_col = []

for c in col_list:
    i = i - 1
    if i == 0:
        current_year = current_year - 1
        current_month = 12
        continue
        
    c = str(current_year) + ' ' + calendar.month_name[i]
    print(c)
    new_col.append(c)
    
print(new_col)

2023 February
2023 January
2022 December
2022 November
2022 October
2022 September
2022 August
2022 July
2022 June
2022 May
2022 April
2022 March
2022 February
['2023 February', '2023 January', '2022 December', '2022 November', '2022 October', '2022 September', '2022 August', '2022 July', '2022 June', '2022 May', '2022 April', '2022 March', '2022 February']


In [56]:
new_col.append('Materials') 
new_col.reverse()
print(new_col)

['Materials', '2022 February', '2022 March', '2022 April', '2022 May', '2022 June', '2022 July', '2022 August', '2022 September', '2022 October', '2022 November', '2022 December', '2023 January', '2023 February']


In [57]:
dfx.columns = new_col

In [58]:
dfx

,Materials,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137
6,Pre-cast concrete products,148,148.5,150.5,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8,178.2
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,140.6,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1,161.9
8,Concrete reinforcing bars (steel),169.7,213.8,240.7,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2,185.8
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c


In [59]:
mats = dfx['Materials']
dfx = dfx[dfx.columns[:0:-1]]
result = pd.concat([mats, dfx], axis=1)
result

,Materials,2023 February,2023 January,2022 December,2022 November,2022 October,2022 September,2022 August,2022 July,2022 June,2022 May,2022 April,2022 March,2022 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",141.7,142.5,142.7,142.7,136.9,139,137.6,137.6,139.5,139.3,132.7,123.8,123.3
2,...,164,164,164,164,161.5,161.5,161.5,162.2,165.3,166.2,137.3,130.4,129.4
3,Bituminous Mixtures based on Natural & Artific...,139.9,140.1,139.5,139.5,141.7,142.3,137.7,135.8,128.8,127.9,126.4,122.3,122.2
4,Cement,143.6,143.3,138.3,134.4,132.6,132.3,131.1,130.7,127.2,124.3,124.3,121.7,119.9
5,Ready-mixed concrete **,137,137,136,134.3,132.1,131.3,129.3,128.8,128.5,126.3,120.2,119.6,117.2
6,Pre-cast concrete products,178.2,172.8,170.2,169.6,168.1,165.3,163.3,163.2,162.2,161.6,150.5,148.5,148
7,"of which : Blocks, bricks, tiles & fl...",161.9,156.1,151.7,150.8,151.4,149.5,148.9,149,147.7,148.3,140.6,138.5,138.2
8,Concrete reinforcing bars (steel),185.8,185.2,199.6,199.9,201.6,216.2,216.7,222,239.8,242.2,240.7,213.8,169.7
9,All Bricks,c,c,c,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,c,c,c,c


In [233]:
URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
r = requests.get(URL) 
soup = BeautifulSoup(r.content, "html.parser")
with open(r'/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/example_data.xlsx', 'wb') as f:
    f.write(r.content)

filenames = ['March']   
year = ['2020']
list_of_urls_found = []
for a in soup.select(','.join('a:contains("{}")'.format(i) for i in filenames)):
        list_of_urls_found.append(a['href'])

for l in list_of_urls_found:
    if re.findall('March.*2020'.lower(), l.lower()):
        print(l)

https://www.gov.uk/government/statistics/building-materials-and-components-statistics-march-2020


In [92]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

In [93]:
# Creating Empty DataFrame for storing compiled data
UK_Data_Masterfile = pd.DataFrame()
UK_Data_Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

In [94]:
# Webpage
URL = 'https://www.data.gov.uk/dataset/75ee36ed-21f7-4d7b-9e7c-f5bf4546145d/monthly-statistics-of-building-materials-and-components'
r = requests.get(URL) 
soup = BeautifulSoup(r.content, "html.parser")
list_of_urls_found = []

# Current Date
current_date = datetime.today()
current_year = int(current_date.strftime('%Y'))
current_month = int(current_date.strftime('%m'))
current_month = current_month - 2 # Account for delay uploading data at beginning of month by data.gov.uk
start_year = current_year

# Naming Convension
file_name = ''

def Data_download():
    global file_name
    file_name = '{}/UK Data/{} {} UK Monthly Statistics of Building Materials and Components.xlsx'.format(download_destination, current_year,calendar.month_name[current_month])
    list_of_urls_found = []
    for a in soup.select(','.join('a:contains("{}")'.format(i) for i in calendar.month_name[current_month])):
            list_of_urls_found.append(a['href'])
    for l in list_of_urls_found:
        if re.findall('{}.*{}'.format(calendar.month_name[current_month], current_year).lower(), l.lower()):
            download_link = l
            print(download_link)
    
    # Download link
    r = requests.get(download_link) 
    with open(file_name, 'wb') as f:
        f.write(r.content)
        
    Format_and_Compile()

    
    
def Format_and_Compile():
    # Define variable to load the wookbook
    wb = openpyxl.load_workbook(file_name)
    sheets = wb.sheetnames
    ws = wb[sheets[3]]
    df = pd.DataFrame(ws.values)
    # Delete filler rows
    rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
    df = df.drop(index=rows_to_delete)
    df.reset_index(drop=True, inplace=True) # Reset the index 
    df = df.drop(df.tail(15).index)
    # Set Column Headers
    df = df.T.set_index(0).T # Assign top row as column headers
    df.columns.values[0] = 'Materials' # Label index column
    df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

    # Change Header Format to YYYY-Month
    global current_month
    global current_year
    curr_month = current_month
    curr_year = current_year
    col_list = [col for col in df.columns[::-1]]
    new_col = []
    for c in col_list:
        curr_month = curr_month - 1
        if curr_month == 0:
            curr_year = curr_year - 1
            curr_month = 12
            continue    
        c = str(curr_year) + ' ' + calendar.month_name[curr_month]
        new_col.append(c)    
    new_col.append('Materials') 
    new_col.reverse()
    df.columns = new_col
    
    mats = df['Materials']
    df_cols = df[df.columns[:0:-1]]
    df = pd.concat([mats, df_cols], axis=1)

    Concat_data(df)
    
def Concat_data(dataframe):
    if current_year == start_year:
        # Update Masterfile using concat to append new data
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
        Masterfile = pd.concat([UK_Data_Masterfile, dataframe])
        Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')
    else:
        # Update Masterfile using concat to append new data
        UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
        Masterfile = pd.concat([UK_Data_Masterfile, dataframe.iloc[:, 1:]], axis=1, join='outer')
        Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

while current_year >= 2022:
    print('{} {}'.format(current_year, calendar.month_name[current_month]))
    Data_download()
    current_year = current_year - 1

2023 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1148775/23-cs4-_Construction_Building_Materials_-_Tables_March_2023.xlsx
2022 March
https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1065392/22-cs4_-_Construction_Building_Materials_-_Tables_March_2022.xlsx


In [88]:
df.iloc[:, 1:].head()

,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February
1,123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137


In [63]:
current_date = datetime.today()
current_year = int(current_date.strftime('%Y'))
current_year

2023

In [ ]:
# Define variable to load the wookbook
wb = openpyxl.load_workbook(file_name)
sheets = wb.sheetnames
ws = wb[sheets[3]]
df = pd.DataFrame(ws.values)
# Delete filler rows
rows_to_delete = [0,1,2,3,4,5,7,8,12,13,19,20,24,25,32,33,41,42,47,48,58]
df = df.drop(index=rows_to_delete)
df.reset_index(drop=True, inplace=True) # Reset the index 
df = df.drop(df.tail(15).index)
# Set Column Headers
df = df.T.set_index(0).T # Assign top row as column headers
df.columns.values[0] = 'Materials' # Label index column
df = df.drop([col for col in df.columns if not col], axis=1) # Delete Filler Columns

# Change Header Format to YYYY-Month
col_list = [col for col in df.columns[::-1]]
new_col = []
for c in col_list:
    current_month = current_month - 1
    if current_month == 0:
        current_year = current_year - 1
        current_month = 12
        continue    
    c = str(current_year) + ' ' + calendar.month_name[current_month]
    new_col.append(c)    
new_col.append('Materials') 
new_col.reverse()
df.columns = new_col
df.head()

In [72]:
x = df
y = df

In [65]:
xy = pd.concat([x, y], axis=1, join='outer')
xy

,Materials,2022 February,2022 March,2022 April,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,...,2022 May,2022 June,2022 July,2022 August,2022 September,2022 October,2022 November,2022 December,2023 January,2023 February
1,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",123.3,123.8,132.7,139.3,139.5,137.6,137.6,139,136.9,...,139.3,139.5,137.6,137.6,139,136.9,142.7,142.7,142.5,141.7
2,...,129.4,130.4,137.3,166.2,165.3,162.2,161.5,161.5,161.5,...,166.2,165.3,162.2,161.5,161.5,161.5,164,164,164,164
3,Bituminous Mixtures based on Natural & Artific...,122.2,122.3,126.4,127.9,128.8,135.8,137.7,142.3,141.7,...,127.9,128.8,135.8,137.7,142.3,141.7,139.5,139.5,140.1,139.9
4,Cement,119.9,121.7,124.3,124.3,127.2,130.7,131.1,132.3,132.6,...,124.3,127.2,130.7,131.1,132.3,132.6,134.4,138.3,143.3,143.6
5,Ready-mixed concrete **,117.2,119.6,120.2,126.3,128.5,128.8,129.3,131.3,132.1,...,126.3,128.5,128.8,129.3,131.3,132.1,134.3,136,137,137
6,Pre-cast concrete products,148,148.5,150.5,161.6,162.2,163.2,163.3,165.3,168.1,...,161.6,162.2,163.2,163.3,165.3,168.1,169.6,170.2,172.8,178.2
7,"of which : Blocks, bricks, tiles & fl...",138.2,138.5,140.6,148.3,147.7,149,148.9,149.5,151.4,...,148.3,147.7,149,148.9,149.5,151.4,150.8,151.7,156.1,161.9
8,Concrete reinforcing bars (steel),169.7,213.8,240.7,242.2,239.8,222,216.7,216.2,201.6,...,242.2,239.8,222,216.7,216.2,201.6,199.9,199.6,185.2,185.8
9,All Bricks,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,c,c
10,Ceramic tiles *,c,c,c,c,c,c,c,c,c,...,c,c,c,c,c,c,c,c,c,c


In [66]:
# Update Masterfile using concat to append new data
UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
Masterfile = pd.concat([UK_Data_Masterfile, x], axis=1, join='outer')
Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')

In [73]:
# Update Masterfile using concat to append new data
UK_Data_Masterfile = pd.read_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv', index_col=0)
Masterfile = pd.concat([UK_Data_Masterfile, y.iloc[:, 1:]], axis=1, join='outer')
Masterfile.to_csv(download_destination + '/UK Data/UK_Building_Materials_CostIndex_Compiled.csv')